In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_lactate_first,lab_lactate_min,lab_lactate_max,lab_lactate_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,0,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1.4,1.4,1.4,0,0,1,0,0.5444486,1.836721
278148,169700,49395,0,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2569337,1.345775
274174,103722,14855,0,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1.6,1.6,2.6,1,0,1,0,0.3264142,3.063592
214954,170515,54642,0,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,3.8,2.2,3.8,1,1,1,0,0.8063102,1.240217
222457,183493,96815,0,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.2239520,1.288580
264413,134244,81436,0,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1.4,1.4,1.4,0,0,1,0,0.3457691,2.892104


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
2.555251e-02,0.04837335,66.68924,65.742830,17.207039,16.554501,66.819125,65.992579,16.751317,16.619096,age
1.571359e-02,0.43360290,50.82285,47.762109,NA,NA,49.269948,48.288405,NA,NA,female
4.654788e-12,0.13519476,78.56295,83.168338,23.579777,26.872532,80.382578,81.361076,24.347537,25.392902,weight
2.009720e-16,0.07229033,8.50597,10.068670,7.451124,7.666592,9.142537,9.484882,7.554606,7.630381,elix_score
1.179391e-15,0.10326054,19.63246,20.764255,5.788347,5.438449,20.020541,20.246750,5.638692,5.421080,saps
3.538666e-28,0.02451590,5.30526,6.332618,3.618029,3.790436,5.678279,5.886310,3.671404,3.702144,sofa


In [5]:
wpv <- table_one %>%
    filter(wtd_p_val < 0.05) %>%
    pull(var) %>%
    gsub("female", "gender", .)

if (length(grep("day", wpv)) > 0) wpv <- wpv %>% discard(grepl("day", .)) %>% c(., "icu_adm_weekday")

wpv

[1] "age"                        "sofa"                      
 [3] "vent"                       "vaso"                      
 [5] "sedative"                   "icd_chf"                   
 [7] "icd_afib"                   "lab_chloride_first"        
 [9] "lab_bnp_flag"               "lab_troponin_flag"         
[11] "lab_creatinine_kinase_flag"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + age + sofa + vent + vaso + sedative + icd_chf + icd_afib + lab_chloride_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -2.504580   0.509142  -4.919 8.91e-07 ***
echo1                       -0.199963   0.071140  -2.811 0.004957 ** 
age                          0.027256   0.002419  11.270  < 2e-16 ***
sofa                         0.222233   0.011315  19.641  < 2e-16 ***
vent1                        0.425563   0.108783   3.912 9.25e-05 ***
vaso1                       -0.076297   0.083232  -0.917 0.359349    
sedative1                   -0.378221   0.108702  -3.479 0.000506 ***
icd_chf1                    -0.122164   0.081617  -1.497 0.134496    
icd_afib1                    0.127511   0.080580   1.582 0.113604    
lab_chloride_first          -0.015540   0.004583  -3.391 0.000702 ***
lab_bnp_flag1                0.01254

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.08170995,0.03012262,0.2216446
echo1,0.81876107,0.71220073,0.9412651
age,1.02763054,1.02277088,1.0325133
sofa,1.24886188,1.22147124,1.2768667
vent1,1.53045137,1.23658263,1.8941568
vaso1,0.92654140,0.78707790,1.0907166
sedative1,0.68507918,0.55362227,0.8477504
icd_chf1,0.88500286,0.75417494,1.0385257
icd_afib1,1.13599721,0.97003602,1.3303523
lab_chloride_first,0.98458006,0.97577532,0.9934642
